In [24]:
import random
import gym
import numpy as np
import collections
from tqdm import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import rl_utils
import itertools
from gym import spaces

In [25]:
class ReplayBuffer:
    ''' 经验回放池 '''
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)  # 队列,先进先出   #定义一个buffer

    def add(self, state, action, reward, next_state, done):  # 将数据加入buffer
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):  # 从buffer中采样数据,数量为batch_size
        transitions = random.sample(self.buffer, batch_size) #通过random.sample函数抽样批量大小的数据到transition中
        state, action, reward, next_state, done = zip(*transitions) #通过zip(*)函数解包(注:解包后是多个元组)得到的批量大小的五元组
        return np.array(state), action, reward, np.array(next_state), done

    def size(self):  # 目前buffer中数据的数量
        return len(self.buffer)

In [26]:
class Qnet(torch.nn.Module):
    ''' 只有一层隐藏层的Q网络 '''
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Qnet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # 隐藏层使用ReLU激活函数
        return self.fc2(x)

In [27]:
class DQN:
    ''' DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma,
                 epsilon, target_update, device):
        self.action_dim = action_dim
        self.q_net = Qnet(state_dim, hidden_dim,
                          self.action_dim).to(device)  # Q网络
        # 目标网络
        self.target_q_net = Qnet(state_dim, hidden_dim,
                                 self.action_dim).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(),
                                          lr=learning_rate)
        self.gamma = gamma  # 折扣因子
        self.epsilon = epsilon  # epsilon-贪婪策略
        self.target_update = target_update  # 目标网络更新频率
        self.count = 0  # 计数器,记录更新次数
        self.device = device

    def take_action(self, state):  # epsilon-贪婪策略采取动作
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.action_dim)
        else:
            state = torch.tensor([state], dtype=torch.float).to(self.device)
            action = self.q_net(state).argmax().item()
        return action

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(
            self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)

        q_values = self.q_net(states).gather(1, actions)  # Q值  动作与价值对应起来
        # 下个状态的最大Q值
        max_next_q_values = self.target_q_net(next_states).max(1)[0].view(
            -1, 1)
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones
                                                                )  # TD误差目标   ## 如果done了那么对应的Q值为0
        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  # 均方误差损失函数
        self.optimizer.zero_grad()  # PyTorch中默认梯度会累积,这里需要显式将梯度置为0--上一次的梯度记录被清空
        dqn_loss.backward()  # 计算梯度
        self.optimizer.step()  #梯度下降法来更新参数的值

        if self.count % self.target_update == 0:
            self.target_q_net.load_state_dict(
                self.q_net.state_dict())  # 更新目标网络
        self.count += 1
        

In [31]:
lr = 2e-3
num_episodes = 500
hidden_dim = 128
gamma = 0.98
epsilon = 0.01
target_update = 10
buffer_size = 10000
minimal_size = 500
batch_size = 64
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

N=4;
M=5;
forms=list(itertools.permutations(range(N, 0, -1)))  #4辆车的全排列 所有可能
formaction = np.array(forms).T  #写成矩阵的形式
numAct=formaction.shape[1]  #转置矩阵的列数，即N的全排列的所有可能数
state=np.array([1,1,1,1,6])
SOC=None
remRsq=None   #所剩余的可重排序地点数
col=None    #col为当下处于第几个重排序地点
steps_beyond_terminated=None        
form_M=None
SOC_M=None
close_t=None
Action = random.choice(range(formaction.shape[1]))  # 选择一个随机的列索引(随机选择一个动作)
form = formaction[:,Action]  # 选择对应的列(选择该动作对应的排列)
        
Delta = np.array([ [0.1302, 0.1334, 0.2522, 0.1868, 0.0787],
                   [0.1224, 0.1255, 0.2372, 0.1756, 0.0741],
                   [0.1170, 0.1199, 0.2266, 0.1678, 0.0708],
                   [0.1170, 0.1199, 0.2266, 0.1678, 0.0708]])
        
#env_name = 'MyEnv-v0'# 定义使用gym库中的某一个环境，'CartPole-v0'可以改为其它环境
#env = gym.make(env_name)
random.seed(0)
np.random.seed(0)
#env.seed(0)
torch.manual_seed(0)
replay_buffer = ReplayBuffer(buffer_size)

observation_space= spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)
state_dim = observation_space.shape[0]  #状态维度

action_space = spaces.Discrete(numAct)
action_dim = action_space.n  #动作维度

def reset(state,SOC,remRsq,col,steps_beyond_terminated,M,N):
        state = np.array([1,1,1,1,6])
        SOC=[]
        for item in range(4):
            SOC.append(float(state[item]))
        remRsq=state[N]   #所剩余的可重排序地点数
        col=M-remRsq+1    #col为当下处于第几个重排序地点
        
        steps_beyond_terminated = None

        return state,SOC,remRsq,col,steps_beyond_terminated
    
def clcSM(SOC,N,Delta):
        SM = SOC.copy()
        for indRsq in range(1):
            for indEV in range(N):
                #print(SM[indEV])
                X=SM[indEV]
                Y=Delta[form[indEV]-1]
                #print(X-Y)
                SM[indEV]=X-Y
        return SM
#最后一个位置进行SOC排序
def socOrderForm(SOC):
        sorted_indices = np.argsort(SOC)
        sorted_SOC = SOC[sorted_indices]
        sorted_input_indices = indices[sorted_indices]
        return sorted_input_indices
    
agent = DQN(state_dim, hidden_dim, action_dim, lr, gamma, epsilon,
            target_update, device)

def step(action,action_space,state,remRsq,N,M,SOC,form_M,SOC_M,cose_t):
        # 检查动作的有效性
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert action_space.contains(action), err_msg
        
        # 确保在使用 step 方法之前已经调用了 reset 方法
        assert state is not None, "Call reset before using step method."
    

        terminated = bool(
            remRsq==1
        )

        if not terminated:
            SOC=clcSM(SOC,N,Delta)   #计算SOC在第一个重排序位置后的SOC值
            remRsq = remRsq - 1;#剩余可重排序位置
            state=np.append(SOC,remRsq)
        else:  # semi-implicit euler
            form_M=socOrderForm(SOC);#最后一次为基于SOC重排序
            SOC_M=clcSM(SOC,form_M,Delta[:,M],N,1)
            state=np.append(SOC_M,remRsq)
            cose_t=1
        
        if not terminated:
            reward = 0
        else:
            reward=-np.std(SOC_M,axis=0)
        
        return state, reward, terminated, {}
    
return_list = []
for i in range(10):#定义的num_episodes为500，这里分为10个iteration,通过进度条显示训练过程#初始化环境
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            state,SOC,remRsq,col,steps_beyond_terminated=reset(state,SOC,remRsq,col,steps_beyond_terminated,M,N)
            done = False
            while not done:
                action = agent.take_action(state)
                next_state, reward, done, _ = step(action,action_space,state,remRsq,N,M,SOC,form_M,SOC_M,close_t)#动作通过step函数得到返回的四元组
                replay_buffer.add(state, action, reward, next_state, done)
                state = next_state
                episode_return += reward
                # 当buffer数据的数量超过一定值后,才进行Q网络训练
                if replay_buffer.size() > minimal_size:
                    b_s, b_a, b_r, b_ns, b_d = replay_buffer.sample(batch_size)
                    transition_dict = {
                        'states': b_s,
                        'actions': b_a,
                        'next_states': b_ns,
                        'rewards': b_r,
                        'dones': b_d
                    }
                    agent.update(transition_dict) #参数更新
            return_list.append(episode_return)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix({
                    'episode':
                    '%d' % (num_episodes / 10 * i + i_episode + 1),
                    'return':
                    '%.3f' % np.mean(return_list[-10:])
                })
            pbar.update(1) #进度加1

Iteration 0:   0%|                                                                              | 0/50 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x21 and 5x128)